In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

from rubin_sim.data import get_data_dir
import sqlite3

from sat_utils import Constellation, starlink_constellation_v1

In [2]:
# load up some visits

limit = 900

dd = get_data_dir()
baseline_file = os.path.join(dd,'sim_baseline/baseline.db')

# Conenct to the sqlite database
con = sqlite3.connect(baseline_file)

# Load up the first year
df = pd.read_sql('select * from observations where night < 1 and sunAlt > -24 ;', con)

con.close()

In [3]:
# data frame with a list of observations
df

,observationId,fieldRA,fieldDec,observationStartMJD,flush_by_mjd,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,...,sunAz,sunRA,sunDec,moonRA,moonDec,moonDistance,solarElong,moonPhase,cummTelAz,scripted_id
0,0,310.024480,-60.812928,60218.001806,60218.023576,30.0,y,-297.249225,-297.249225,2,...,255.593220,186.644048,-2.870827,27.609463,11.956111,94.490314,102.958651,87.407902,169.454444,0
1,1,310.601871,-63.561425,60218.002254,60218.023576,30.0,y,-297.708278,-297.708278,2,...,255.500445,186.644453,-2.871001,27.615338,11.959438,95.029204,101.743959,87.404494,170.502875,0
2,2,311.292611,-66.317774,60218.002703,60218.023576,30.0,y,-297.909620,-297.909620,2,...,255.407493,186.644858,-2.871176,27.621208,11.962765,95.563446,100.497860,87.401088,171.406738,0
3,3,312.140731,-69.082666,60218.003152,60218.023576,30.0,y,-297.838337,-297.838337,2,...,255.314364,186.645264,-2.871350,27.627073,11.966093,96.092842,99.221261,87.397685,172.197791,0
4,4,304.170163,-73.375442,60218.003623,60218.023576,30.0,y,-309.290623,-309.290623,2,...,255.216260,186.645690,-2.871533,27.633232,11.969593,99.605370,94.821142,87.394110,177.239803,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,830,336.319706,-85.448079,60218.392486,60218.404999,30.0,i,-56.043578,-56.043578,2,...,101.469398,186.997138,-3.022386,31.259086,14.500684,101.856712,90.893919,85.254586,146.948266,0
119,831,294.594225,-87.438033,60218.393053,60218.404999,30.0,i,-96.112005,-96.112005,2,...,101.360003,186.997650,-3.022606,31.264624,14.503548,104.786408,87.754422,85.251529,143.526699,0
120,832,324.991600,-82.868303,60218.393578,60218.404999,30.0,i,-68.489767,-68.489767,2,...,101.258784,186.998125,-3.022809,31.269767,14.506202,101.548293,92.280883,85.248692,149.115181,0
121,833,302.751653,-81.835351,60218.394084,60218.404999,30.0,i,-90.162716,-90.162716,2,...,101.161561,186.998582,-3.023006,31.274722,14.508754,104.149030,90.539605,85.245958,147.872290,0


In [4]:
# Make a satellite constellation
tles = starlink_constellation_v1()
constellation = Constellation(tles)

In [5]:
# Check the first method
lengths = []
nstreaks = []
n_rows = len(df)
t1 = time.time()
for index, row in df.iterrows():
    length, streak = constellation.check_pointing(row['altitude'], row['azimuth'],
                                    row['observationStartMJD'], row['visitTime'])
    lengths.append(length)
    nstreaks.append(streak)
    # A simple progress bar
    progress = index/float(n_rows)*100
    text = "\rprogress = %.3f%%" % progress
    sys.stdout.write(text) 
    sys.stdout.flush() 
t2 = time.time()
print('runtime = %.2f min' % ((t2-t1)/60.) )

progress = 99.187%runtime = 17.55 min


In [18]:
# check the new method
t1 = time.time()

fast_lengths, fast_nstreaks = constellation.check_pointings(df['fieldRA'].values, df['fieldDec'].values,
                                                           df['observationStartMJD'].values,
                                                           df['visitTime'].values)
t2 = time.time()
print('runtime = %.2f min' % ((t2-t1)/60.) )


runtime = 0.22 min


In [19]:
np.array(lengths)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 3.45790382, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 3.35728576, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.48420194, 0.        , 3.01488367,
       0.        , 3.94327343, 3.49311862, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 2.65747914, 3.49479904, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 4.76254

In [21]:
fast_lengths

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 3.46212246, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 3.4627139 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.39851521, 0.        , 3.10156612,
       0.        , 4.07961247, 3.46794492, 1.46467336, 3.37318255,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       3.40650719, 0.        , 2.7307857 , 3.49614536, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 4.71422

In [23]:
(fast_lengths-np.array(lengths))/fast_lengths

/var/folders/05/z_zc9f654c5dlw06t4s_d68w0000gn/T/ipykernel_21285/2006847437.py:1: RuntimeWarning: invalid value encountered in true_divide
  (fast_lengths-np.array(lengths))/fast_lengths


array([            nan,             nan,             nan,             nan,
                   nan,             nan,  1.21851056e-03,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,  3.04466799e-02,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan, -2.15014956e-01,
                   nan,  2.79479616e-02,             nan,  3.34196069e-02,
       -7.25896683e-03,  

In [11]:
fast_lengths, fast_nstreaks

(array([0.]), array([0]))

In [12]:
constellation.check_pointing(df['altitude'].values[i], df['azimuth'].values[i], df['observationStartMJD'].values[i],
                                                           df['visitTime'].values[i])

(0.0, 0)

In [13]:
# So, why is the fast method finding more streaks than the slow method?